In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
    

/kaggle/input/kaggle-kl-div/kaggle_metric_utilities.py
/kaggle/input/kaggle-kl-div/kaggle_kl_div.py
/kaggle/input/specs-fr-eegs/true_data_Inception.csv
/kaggle/input/specs-fr-eegs/oof_effNet.csv
/kaggle/input/specs-fr-eegs/oof_Inception.csv
/kaggle/input/specs-fr-eegs/hms_data_generator.py
/kaggle/input/specs-fr-eegs/oof_ResNet.csv
/kaggle/input/specs-fr-eegs/true_data_MobileNet.csv
/kaggle/input/specs-fr-eegs/true_data_effNet.csv
/kaggle/input/specs-fr-eegs/oof_MobileNet.csv
/kaggle/input/specs-fr-eegs/spec_eeg.py
/kaggle/input/specs-fr-eegs/true_data_ResNet.csv


In [2]:
import sys
sys.path.append('/kaggle/input/kaggle-kl-div')
from kaggle_kl_div import score

In [3]:
TARGETS = ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']

# Load original data and calculate the KL-divergence score

In [4]:
oof_I = pd.read_csv('/kaggle/input/specs-fr-eegs/oof_Inception.csv')
oof_M = pd.read_csv('/kaggle/input/specs-fr-eegs/oof_MobileNet.csv')
oof_e = pd.read_csv('/kaggle/input/specs-fr-eegs/oof_effNet.csv')
oof_R = pd.read_csv('/kaggle/input/specs-fr-eegs/oof_ResNet.csv')

true_I = pd.read_csv('/kaggle/input/specs-fr-eegs/true_data_Inception.csv')
true_M = pd.read_csv('/kaggle/input/specs-fr-eegs/true_data_MobileNet.csv')
true_e = pd.read_csv('/kaggle/input/specs-fr-eegs/true_data_effNet.csv')
true_R = pd.read_csv('/kaggle/input/specs-fr-eegs/true_data_ResNet.csv')

In [5]:
def array_to_df(array):
    return pd.DataFrame(array, columns=[f'{TARGETS[i]}' for i in range(6)])

def score_KL(oof_ens, target):
    oof_ens_c = pd.DataFrame(oof_ens.copy())
    oof_ens_c['id'] = np.arange(len(oof_ens_c))

    target_c = pd.DataFrame(target.copy())
    target_c['id'] =  np.arange(len(target_c))

    score_kl = score(solution=target_c, submission=oof_ens_c, row_id_column_name='id')
    return score_kl

In [6]:
print(f'KL-div score of InceptionV3 is {score_KL(oof_I, true_I)}')
print(f'KL-div score of EffNetB0 is {score_KL(oof_e, true_e)}')
print(f'KL-div score of ResNet50 is {score_KL(oof_R, true_R)}')
print(f'KL-div score of MobileNetV2 is {score_KL(oof_M, true_M)}')

KL-div score of InceptionV3 is 0.6227407926346561
KL-div score of EffNetB0 is 0.4352441838155703
KL-div score of ResNet50 is 0.7277101380351932
KL-div score of MobileNetV2 is 0.7651174237089369


In [7]:
#CONVERT DATAFRAME TO NUMPY FOR HILL CLIMBING
#ALL 'TRUE' DF ARE THE SAME
trueIn = true_I.to_numpy()
trueMn = true_M.to_numpy()
trueRn = true_R.to_numpy()
trueEn = true_e.to_numpy()

np.array_equal(trueIn, trueMn), np.array_equal(trueRn, trueMn), np.array_equal(trueEn, trueMn)

(True, True, True)

In [8]:
#this is the target dataframe, which stays the same
true = pd.DataFrame(trueIn, columns=[f'{TARGETS[i]}' for i in range(6)])
true

,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,0.0,0.0,0.0000,0.0,0.0,1.0000
1,0.0,0.0,0.0000,0.0,0.0,1.0000
2,1.0,0.0,0.0000,0.0,0.0,0.0000
3,0.0,0.0,0.0000,0.0,0.0,1.0000
4,0.0,0.0,0.0000,1.0,0.0,0.0000
...,...,...,...,...,...,...
17084,0.0,0.0,0.0000,0.0,0.0,1.0000
17085,0.0,0.0,0.0625,0.0,0.5,0.4375
17086,1.0,0.0,0.0000,0.0,0.0,0.0000
17087,0.0,0.0,0.0000,0.0,0.5,0.5000


In [9]:
#use the original dataframes to convert to numpy, not the ones passed into 'score'
oofIn = oof_I.to_numpy()
oofRn = oof_R.to_numpy()
oofMn = oof_M.to_numpy()
oofEn = oof_e.to_numpy()


In [10]:
oof_e

,EffNet_seizure_vote,EffNet_lpd_vote,EffNet_gpd_vote,EffNet_lrda_vote,EffNet_grda_vote,EffNet_other_vote
0,0.015231,0.026490,0.050624,0.091274,0.301535,0.514845
1,0.001306,0.001227,0.000380,0.022690,0.037037,0.937361
2,0.960015,0.002344,0.021974,0.002062,0.003240,0.010365
3,0.001353,0.011533,0.001263,0.003461,0.001519,0.980871
4,0.015509,0.391170,0.003174,0.282850,0.024089,0.283210
...,...,...,...,...,...,...
17084,0.009303,0.003682,0.001667,0.095471,0.235951,0.653925
17085,0.001356,0.039365,0.111657,0.015102,0.175335,0.657186
17086,0.876966,0.024205,0.008216,0.033995,0.016672,0.039945
17087,0.001363,0.008163,0.002240,0.060099,0.073802,0.854333


In [11]:
oof_I

,Inception_seizure_vote,Inception_lpd_vote,Inception_gpd_vote,Inception_lrda_vote,Inception_grda_vote,Inception_other_vote
0,0.126335,0.065561,0.015355,0.134317,0.093141,0.565290
1,0.001828,0.001204,0.000411,0.030388,0.162758,0.803411
2,0.605066,0.103199,0.087568,0.025624,0.024876,0.153668
3,0.004919,0.021849,0.001858,0.015330,0.011133,0.944911
4,0.037646,0.652729,0.002716,0.178015,0.008345,0.120548
...,...,...,...,...,...,...
17084,0.025531,0.007660,0.002583,0.107177,0.427939,0.429109
17085,0.005887,0.092412,0.045702,0.080737,0.273332,0.501931
17086,0.375360,0.056309,0.089752,0.048410,0.178991,0.251178
17087,0.005325,0.014737,0.001271,0.059799,0.052161,0.866707


In [12]:
oof_M

,MobNet_seizure_vote,MobNet_lpd_vote,MobNet_gpd_vote,MobNet_lrda_vote,MobNet_grda_vote,MobNet_other_vote
0,0.031288,0.009015,0.018548,0.049050,0.440504,0.451595
1,0.000375,0.002690,0.001378,0.021973,0.037674,0.935910
2,0.879055,0.002317,0.037832,0.005083,0.037732,0.037981
3,0.003552,0.012988,0.011099,0.018009,0.045244,0.909108
4,0.013279,0.155415,0.003276,0.364367,0.036656,0.427007
...,...,...,...,...,...,...
17084,0.014503,0.002679,0.001379,0.066759,0.783959,0.130720
17085,0.041616,0.087458,0.065350,0.074852,0.191740,0.538985
17086,0.554087,0.019505,0.007376,0.034189,0.018930,0.365914
17087,0.005769,0.000596,0.001291,0.008719,0.211701,0.771924


In [13]:
true_I

,Inception_seizure_vote,Inception_lpd_vote,Inception_gpd_vote,Inception_lrda_vote,Inception_grda_vote,Inception_other_vote
0,0.0,0.0,0.0000,0.0,0.0,1.0000
1,0.0,0.0,0.0000,0.0,0.0,1.0000
2,1.0,0.0,0.0000,0.0,0.0,0.0000
3,0.0,0.0,0.0000,0.0,0.0,1.0000
4,0.0,0.0,0.0000,1.0,0.0,0.0000
...,...,...,...,...,...,...
17084,0.0,0.0,0.0000,0.0,0.0,1.0000
17085,0.0,0.0,0.0625,0.0,0.5,0.4375
17086,1.0,0.0,0.0000,0.0,0.0,0.0000
17087,0.0,0.0,0.0000,0.0,0.5,0.5000


# Test some combinations of ensemble

In [30]:
def hc(current_best_ensemble,  MODELS):
    # Initialise
    STOP = False
    #current_best_ensemble = oofMn

    #MODELS = np.array([oofIn])
    weight_range = np.arange(0.01,0.99,0.01)   
    history = [score_KL(array_to_df(current_best_ensemble), true)]
    i=0

    # Hill climbing
    while not STOP:
        i+=1
        print(f'Iteration {i}')
        potential_best_score = score_KL(array_to_df(current_best_ensemble), true)
        k_best, wgt_best = None, None
        for k in range(len(MODELS)):
            for wgt in weight_range:
                potential_ensemble = (1-wgt) * current_best_ensemble + wgt * MODELS[k]
                new_score = score_KL(array_to_df(potential_ensemble),true)
                if new_score < potential_best_score:
                    potential_best_score = new_score
                    k_best, wgt_best = k, wgt
                    print(f'Best k={k_best}, best weight = {wgt_best: .2f}, best KL score {potential_best_score:.5f}')
        if k_best is not None:
            current_best_ensemble = (1-wgt_best) * current_best_ensemble + wgt_best * MODELS[k_best]
            print(f'Iteration: {i}, Model added: {k_best}, Best weight: {wgt_best:.2f}, Best score: {potential_best_score:.5f}')
            MODELS=np.delete(MODELS, k_best, 0)
            if MODELS.shape[0]==0:
                STOP = True
            history.append(potential_best_score)
        else:
            STOP = True
        STOP = True

In [14]:
weight_range = np.arange(0.01,0.99,0.01)
weight_range

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98])

# HC: MobNet + Inception

In [24]:
hc(oofMn, [oofIn])

Iteration 1
Best k=0, best weight =  0.01, best KL score 0.75453
Best k=0, best weight =  0.02, best KL score 0.74620
Best k=0, best weight =  0.03, best KL score 0.73893
Best k=0, best weight =  0.04, best KL score 0.73239
Best k=0, best weight =  0.05, best KL score 0.72640
Best k=0, best weight =  0.06, best KL score 0.72085
Best k=0, best weight =  0.07, best KL score 0.71568
Best k=0, best weight =  0.08, best KL score 0.71082
Best k=0, best weight =  0.09, best KL score 0.70623
Best k=0, best weight =  0.10, best KL score 0.70188
Best k=0, best weight =  0.11, best KL score 0.69776
Best k=0, best weight =  0.12, best KL score 0.69382
Best k=0, best weight =  0.13, best KL score 0.69007
Best k=0, best weight =  0.14, best KL score 0.68648
Best k=0, best weight =  0.15, best KL score 0.68304
Best k=0, best weight =  0.16, best KL score 0.67974
Best k=0, best weight =  0.17, best KL score 0.67657
Best k=0, best weight =  0.18, best KL score 0.67353
Best k=0, best weight =  0.19, bes

In [33]:
score_KL(array_to_df(0.26*oofMn + 0.74*oofIn), true)

0.6051335952707193

## HC[HC[MobNet, Inc] + ResNet]

In [34]:
hc(0.26*oofMn + 0.74*oofIn, np.array([oofRn]))

Iteration 1
Best k=0, best weight =  0.01, best KL score 0.60424
Best k=0, best weight =  0.02, best KL score 0.60347
Best k=0, best weight =  0.03, best KL score 0.60279
Best k=0, best weight =  0.04, best KL score 0.60221
Best k=0, best weight =  0.05, best KL score 0.60169
Best k=0, best weight =  0.06, best KL score 0.60125
Best k=0, best weight =  0.07, best KL score 0.60086
Best k=0, best weight =  0.08, best KL score 0.60054
Best k=0, best weight =  0.09, best KL score 0.60027
Best k=0, best weight =  0.10, best KL score 0.60005
Best k=0, best weight =  0.11, best KL score 0.59988
Best k=0, best weight =  0.12, best KL score 0.59976
Best k=0, best weight =  0.13, best KL score 0.59968
Best k=0, best weight =  0.14, best KL score 0.59965
Iteration: 1, Model added: 0, Best weight: 0.14, Best score: 0.59965


In [43]:
cbes1 = 0.86*(0.26*oofMn + 0.74*oofIn)+0.14*oofRn
score_KL(array_to_df(cbes1), true)

0.5996478244707993

## HC(((MobNet,Inc), ResNet), EffNet)

In [38]:
hc(cbes1, np.array([oofEn]))

Iteration 1
Best k=0, best weight =  0.01, best KL score 0.59346
Best k=0, best weight =  0.02, best KL score 0.58812
Best k=0, best weight =  0.03, best KL score 0.58325
Best k=0, best weight =  0.04, best KL score 0.57872
Best k=0, best weight =  0.05, best KL score 0.57445
Best k=0, best weight =  0.06, best KL score 0.57040
Best k=0, best weight =  0.07, best KL score 0.56653
Best k=0, best weight =  0.08, best KL score 0.56282
Best k=0, best weight =  0.09, best KL score 0.55925
Best k=0, best weight =  0.10, best KL score 0.55580
Best k=0, best weight =  0.11, best KL score 0.55247
Best k=0, best weight =  0.12, best KL score 0.54924
Best k=0, best weight =  0.13, best KL score 0.54611
Best k=0, best weight =  0.14, best KL score 0.54306
Best k=0, best weight =  0.15, best KL score 0.54010
Best k=0, best weight =  0.16, best KL score 0.53721
Best k=0, best weight =  0.17, best KL score 0.53440
Best k=0, best weight =  0.18, best KL score 0.53166
Best k=0, best weight =  0.19, bes

In [44]:
cbes1b = 0.06*(0.86*(0.26*oofMn + 0.74*oofIn)+0.14*oofRn) + 0.94*oofEn

In [45]:
score_KL(array_to_df(cbes1b), true)

0.4334540032213935

# HC starting with HC[MobNet, ResNet]

In [25]:
hc(oofMn, np.array([oofRn]))

Iteration 1
Best k=0, best weight =  0.01, best KL score 0.75315
Best k=0, best weight =  0.02, best KL score 0.74450
Best k=0, best weight =  0.03, best KL score 0.73724
Best k=0, best weight =  0.04, best KL score 0.73087
Best k=0, best weight =  0.05, best KL score 0.72517
Best k=0, best weight =  0.06, best KL score 0.71999
Best k=0, best weight =  0.07, best KL score 0.71523
Best k=0, best weight =  0.08, best KL score 0.71084
Best k=0, best weight =  0.09, best KL score 0.70676
Best k=0, best weight =  0.10, best KL score 0.70295
Best k=0, best weight =  0.11, best KL score 0.69938
Best k=0, best weight =  0.12, best KL score 0.69603
Best k=0, best weight =  0.13, best KL score 0.69287
Best k=0, best weight =  0.14, best KL score 0.68990
Best k=0, best weight =  0.15, best KL score 0.68710
Best k=0, best weight =  0.16, best KL score 0.68445
Best k=0, best weight =  0.17, best KL score 0.68194
Best k=0, best weight =  0.18, best KL score 0.67956
Best k=0, best weight =  0.19, bes

## HC[(0.5MobNet+0.5ResNet), Inception]

In [26]:
hc(0.5*oofRn+ 0.5*oofMn, np.array([oofIn]))

Iteration 1
Best k=0, best weight =  0.01, best KL score 0.64620
Best k=0, best weight =  0.02, best KL score 0.64422
Best k=0, best weight =  0.03, best KL score 0.64236
Best k=0, best weight =  0.04, best KL score 0.64058
Best k=0, best weight =  0.05, best KL score 0.63888
Best k=0, best weight =  0.06, best KL score 0.63725
Best k=0, best weight =  0.07, best KL score 0.63568
Best k=0, best weight =  0.08, best KL score 0.63417
Best k=0, best weight =  0.09, best KL score 0.63271
Best k=0, best weight =  0.10, best KL score 0.63131
Best k=0, best weight =  0.11, best KL score 0.62995
Best k=0, best weight =  0.12, best KL score 0.62864
Best k=0, best weight =  0.13, best KL score 0.62737
Best k=0, best weight =  0.14, best KL score 0.62615
Best k=0, best weight =  0.15, best KL score 0.62496
Best k=0, best weight =  0.16, best KL score 0.62381
Best k=0, best weight =  0.17, best KL score 0.62270
Best k=0, best weight =  0.18, best KL score 0.62162
Best k=0, best weight =  0.19, bes

In [18]:
#Best weight: 0.61, Best score: 0.60032
oof_ens = 0.39*(0.5*oofRn + 0.5*oofMn) + 0.61*oofIn
score_KL(array_to_df(oof_ens), true)

0.6003241318364761

## HC[HC[(0.5MobNet+0.5ResNet), Inception],EffNet]

In [19]:
# Initialise
STOP = False
current_best_ensemble = 0.39*(0.5*oofRn + 0.5*oofMn) + 0.61*oofIn

MODELS = np.array([oofEn])
weight_range = np.arange(0.01,0.99,0.01)   
history = [score_KL(array_to_df(current_best_ensemble), true)]
i=0

# Hill climbing
while not STOP:
    i+=1
    print(f'Iteration {i}')
    potential_best_score = score_KL(array_to_df(current_best_ensemble), true)
    k_best, wgt_best = None, None
    for k in range(len(MODELS)):
        for wgt in weight_range:
            potential_ensemble = (1-wgt) * current_best_ensemble + wgt * MODELS[k]
            new_score = score_KL(array_to_df(potential_ensemble),true)
            if new_score < potential_best_score:
                potential_best_score = new_score
                k_best, wgt_best = k, wgt
                print(f'Best k={k_best}, best weight = {wgt_best: .2f}, best KL score {potential_best_score:.5f}')
    if k_best is not None:
        current_best_ensemble = (1-wgt_best) * current_best_ensemble + wgt_best * MODELS[k_best]
        print(f'Iteration: {i}, Model added: {k_best}, Best weight: {wgt_best:.2f}, Best score: {potential_best_score:.5f}')
        MODELS=np.delete(MODELS, k_best, 0)
        if MODELS.shape[0]==0:
            STOP = True
        history.append(potential_best_score)
    else:
        STOP = True
    STOP = True

Iteration 1
Best k=0, best weight =  0.01, best KL score 0.59429
Best k=0, best weight =  0.02, best KL score 0.58902
Best k=0, best weight =  0.03, best KL score 0.58420
Best k=0, best weight =  0.04, best KL score 0.57971
Best k=0, best weight =  0.05, best KL score 0.57547
Best k=0, best weight =  0.06, best KL score 0.57144
Best k=0, best weight =  0.07, best KL score 0.56759
Best k=0, best weight =  0.08, best KL score 0.56389
Best k=0, best weight =  0.09, best KL score 0.56033
Best k=0, best weight =  0.10, best KL score 0.55689
Best k=0, best weight =  0.11, best KL score 0.55357
Best k=0, best weight =  0.12, best KL score 0.55034
Best k=0, best weight =  0.13, best KL score 0.54721
Best k=0, best weight =  0.14, best KL score 0.54416
Best k=0, best weight =  0.15, best KL score 0.54120
Best k=0, best weight =  0.16, best KL score 0.53831
Best k=0, best weight =  0.17, best KL score 0.53549
Best k=0, best weight =  0.18, best KL score 0.53274
Best k=0, best weight =  0.19, bes

In [20]:
#Best weight: 0.94, Best score: 0.43351
oof_ens1 = 0.39*(0.5*oofRn + 0.5*oofMn) + 0.61*oofIn
oof_ens = 0.06*oof_ens1 + 0.94*oofEn
score_KL(array_to_df(oof_ens), true)

0.43351042516921295

In [21]:
score_KL(array_to_df(oofEn), true)

0.4352441838155703